In [42]:
from bs4 import BeautifulSoup
from urllib.request import Request, urlopen
import pandas as pd
import re
import itertools as it

In [43]:
counties = ['Noble']
url = 'https://www.in.gov/counties/noble/election-info/historical-election-info/2024-general-election-by-precinct/'

In [44]:
i = 0
county = counties[i]
print(county)

Noble


In [45]:
output_file = "../data/in_2024_results/" + county + ".csv"
output_2 = "C:/Users/johnsor/Documents/openelections-data-in/2024/counties/20241105__in__general__" + str.lower(county) + "__precinct.csv"
output_2

'C:/Users/johnsor/Documents/openelections-data-in/2024/counties/20241105__in__general__noble__precinct.csv'

In [46]:
req = Request(url)
html_page = urlopen(req).read()
soup = BeautifulSoup(html_page, 'html.parser')
data = soup.find('section', {'id' : 'content_container_864015'})

In [47]:
cols = {'county' : [],
        'precinct' : [],
        'office' : [],
        'district' : [],
        'party' : [],
        'candidate' : [],
        'election_day' : [],
        'absentee' : [],
        'early_voting' : [],
        'votes' : []}
party_convert = {'R' : 'REP',
                 'REP' : 'REP',
                 'D' : 'DEM',
                 'DEM' : 'DEM',
                 'L' : 'LIB',
                 'LIB' : 'LIB',
                 'WTP' : 'WTP',
                 'W' : 'WTP',
                 'NP' : '',
                 'I' : 'IND',
                 'IND' : 'IND'
                }

In [48]:

for div in data.find_all('div'):
    if div.attrs['class'] == ['s5_']:
        election_day = div.find('span', {'class' : 'f91_'}).text
        absentee = div.find('span', {'class' : 'f90_'}).text
        early_voting = div.find('span', {'class' : 'f89_'}).text
        votes = div.find('span', {'class' : 'f92_'}).text
        registered = div.find('span', {'class' : 'f100_'}).text
    elif div.attrs['class'] == ['s7_']:
        precinct = re.search('Precinct Name: (.*)',
            div.find('span', {'class' : 'f42_'}).text).group(1)
        cols['county'].extend([county, county])
        cols['precinct'].extend([precinct, precinct])
        cols['office'].extend(['Ballots Cast', 'Registered Voters'])
        cols['district'].extend(['',''])
        cols['party'].extend(['',''])
        cols['candidate'].extend(['',''])
        cols['election_day'].extend([election_day,''])
        cols['absentee'].extend([absentee,''])
        cols['early_voting'].extend([early_voting,''])
        cols['votes'].extend([votes, registered])
    elif div.attrs['class'] == ['s9_']:
        office = div.find('span', {'class' : 'f0_'}).text
        if re.search(r'Senator District (\d+)|Representative District (\d+)', office):
            office, district = office.split(' District ')
        else:
            district = ''
    elif div.attrs['class'] == ['s0_']:
        election_day = div.find('span', {'class' : 'f13_'}).text
        absentee = div.find('span', {'class' : 'f12_'}).text
        early_voting = div.find('span', {'class' : 'f11_'}).text
        votes = div.find('span', {'class' : 'f10_'}).text
        candidate = div.find('span', {'class' : 'f8_'}).text
        if re.search(r'\(\w+\)', candidate):
            party = party_convert[
                re.findall(r'\((\w+)\)', candidate)[0]
            ]
            candidate = re.findall(r'\(\w+\) (.*)', candidate)[0]
        else:
            party = ''
        cols['county'].append(county)
        cols['precinct'].append(precinct)
        cols['office'].append(office)
        cols['district'].append(district)
        cols['party'].append(party)
        cols['candidate'].append(candidate)
        cols['election_day'].append(election_day)
        cols['absentee'].append(absentee)
        cols['early_voting'].append(early_voting)
        cols['votes'].append(votes)

In [49]:
df = pd.DataFrame(data=cols)
df

,county,precinct,office,district,party,candidate,election_day,absentee,early_voting,votes
0,Noble,Washington 1,Ballots Cast,,,,259,24,247,530
1,Noble,Washington 1,Registered Voters,,,,,,,811
2,Noble,Washington 1,Public Question-Const Amendment,,,Yes,132,12,109,253
3,Noble,Washington 1,Public Question-Const Amendment,,,No,105,8,107,220
4,Noble,Washington 1,President and Vice-President of the U.S.,,REP,Trump \ Vance,203,15,192,410
...,...,...,...,...,...,...,...,...,...,...
1502,Noble,Noble 29,IN Supreme Court-Massa,,,No,66,6,57,129
1503,Noble,Noble 29,IN Supreme Court-Molter,,,Yes,200,19,174,393
1504,Noble,Noble 29,IN Supreme Court-Molter,,,No,63,3,52,118
1505,Noble,Noble 29,Court of Appeals Dist 4-Pyle III,,,Yes,204,18,173,395


In [50]:
df['precinct']

0       Washington 1
1       Washington 1
2       Washington 1
3       Washington 1
4       Washington 1
            ...     
1502        Noble 29
1503        Noble 29
1504        Noble 29
1505        Noble 29
1506        Noble 29
Name: precinct, Length: 1507, dtype: object

In [51]:
df.to_csv(output_file)

In [52]:
df.to_csv(output_2, index=False, encoding='utf-8')